In [1]:
# Import Needed Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pathlib
import random
import math
import splitfolders


In [2]:
for dir_name, _, filenames in os.walk("/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food"):
    for filename in filenames:
        print(os.path.join(dir_name, filename))

/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (204) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (86) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (101) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (25) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (171) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/train_image_png_115.png
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/train_image_png_101.png
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (1) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/train_image_png_129.png
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food (55) (Copy).jpg
/Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/train_image_png_11.p

In [3]:
%mkdir food_not_food_dir
%cd food_not_food_dir
%mkdir Food NotFood

%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/food*.jpg Food/
%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/food_not_food/data/train_image_png_*.png NotFood/

%ls

/Users/rish/Desktop/Mini-Projects/MLProjects/food_not_food_dir
Food/    NotFood/


In [3]:
epochs = 6
batch_size = 16
img_hieght, img_width = 300, 300
input_shape = (img_hieght, img_width, 3)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
def create_data_binary(data_bs):
    data_bs = pathlib.Path(data_bs)
    splitfolders.ratio(data_bs, output="FoodNotFoodIMG/", seed = 42, ratio = (0.7, 0.15, 0.15), group_prefix=None)
    data_gen = ImageDataGenerator(rescale=1.0/255)
    train_ds = data_gen.flow_from_directory("FoodNotFoodIMG/train/", target_size = (img_hieght, img_width), class_mode = 'binary', 
                                            batch_size = batch_size, subset = "training")
    val_ds = data_gen.flow_from_directory("FoodNotFoodIMG/val/", target_size = (img_hieght, img_width), class_mode = 'binary', 
                                            batch_size = batch_size, shuffle = False)
    test_ds = data_gen.flow_from_directory("FoodNotFoodIMG/test", target_size = (img_hieght, img_width), class_mode = 'binary',
                                            batch_size = batch_size, shuffle = False)
    
    return train_ds, val_ds, test_ds


In [7]:
train_ds, val_ds, test_ds = create_data_binary("/Users/rish/Desktop/Mini-Projects/MLProjects/food_not_food_dir")

Copying files: 408 files [00:00, 999.15 files/s] 

Found 284 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


# Normal Model (mixed precision ON)

In [15]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu", input_shape = input_shape),
    tf.keras.layers.Conv2D(16, (2,2), activation = "relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(100, activation = "relu"),
    tf.keras.layers.Dense(30, activation = "relu"),

    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [16]:
model_1.compile(loss = "binary_crossentropy", optimizer = "adam", 
metrics = ["accuracy"])

In [17]:
model_1.fit(train_ds, batch_size = batch_size, epochs = epochs)

Epoch 1/6
18/18 [==============================] - 584s 32s/step - loss: 3.0242 - accuracy: 0.5775
Epoch 2/6
18/18 [==============================] - 569s 32s/step - loss: 0.3337 - accuracy: 0.8768
Epoch 3/6
17/18 [===========================>..] - ETA: 31s - loss: 0.1794 - accuracy: 0.9328 

KeyboardInterrupt: 